# Task02
Needs Blackhole configured

In [35]:
# Import requirements
import spotipy
import pandas
import random
import numpy
import os

In [36]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 44100  # Sample rate
seconds = 24  # Duration of recording

In [37]:
# Import specific functions
from lib import *

In [38]:
# subject name/id
subject_id = 'sub-19'

In [39]:
# Setup Spotipy
scope = "user-library-read,user-read-playback-state,user-modify-playback-state"
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(scope=scope,client_id='253a1345850d4a5c94995c1159d286e4',
                                             client_secret='9a03e1bcb8124eed9b661991e84bf477',
                                             redirect_uri='http://127.0.0.1:9090'),
)

# choose deviceID (1st)
res = sp.devices()
devID = res['devices'][0]['id']

## Import subject-specific selection of songs

In [40]:
# read file
D = pandas.read_table(os.path.join('..','data','music_matrices',
                                   f"music_{subject_id}.tsv"),
                                index_col = "Emotion")

# extract list of emotions
emotionList = D.index.values

# Generate two new columns (start1MS and start2MS) in milliseconds for each song
for emotion in emotionList:
    D.loc[emotion,"startMS1"] = (int(D.start1[emotion][1:3])*60 + int(D.start1[emotion][4:6])) * 1000
    D.loc[emotion,"startMS2"] = (int(D.start2[emotion][1:3])*60 + int(D.start2[emotion][4:6])) * 1000

## Version 0.2a - 2 songs coupled by emotion

In [41]:
run_id = 'task-02a_run-1'

# create a new list with the same elements as emotionList, but in a random order
randomEmotionList = emotionList.copy()
random.shuffle(randomEmotionList)

# Initialize Dataframe to record timestamps
timeMatrix = pandas.DataFrame(columns=["emotion","trackID","timestamp0","before_music_start", "after_music_start", "before_noise_start", "after_noise_start"])

# set initial volume
sp.volume(100,devID)

# first baseline
timestamp0 = time.time()
#sp.start_playback(uris=['spotify:track:7Ipe9FfRceDN51YMyd99pO'],position_ms=1000,device_id=devID)
#wait_until(timestamp0 + 2)

# trial loop
for emotion in randomEmotionList:
    print(f"Now playing: {emotion}")

    for i in range(1,3):
        song = D[f"trackID{str(i)}"][emotion]
        start_time = D[f"startMS{str(i)}"][emotion].item()

        print(f"Now playing: {song} starting at {str(start_time)}")

        timestamp1 = time.time()

        sp.start_playback(
            uris=[f'spotify:track:{song}'],
            position_ms=start_time,
            device_id=devID,
        )

        timestamp2 = time.time()

        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)

        sd.wait()  # Wait until recording is finished
        
        sp.pause_playback()

        write(os.path.join('..','data','wav_files',
                           f'{subject_id}_{emotion.replace(" ", "")}_{song}.wav'),
              fs, myrecording)  # Save as WAV file

        timestamp3 = time.time()

        wait_until(timestamp3 + 2)

print('Done!')

Now playing: Nostalgia
Now playing: 3lAqHOGFBABytXxiPpDm6e starting at 90000.0
Now playing: 1Lom3Qqs0HUPxqnmRVumKQ starting at 3000.0
Now playing: Peacefulness
Now playing: 6Er8Fz6fuZNi5cvwQjv1ya starting at 32000.0
Now playing: 2agBDIr9MYDUducQPC1sFU starting at 90000.0
Now playing: Tenderness
Now playing: 7LRzgZuP7H12oSC1CnC2jB starting at 42000.0
Now playing: 5lm18pjbwdth6ENVllxjfl starting at 25000.0
Now playing: Tension
Now playing: 3Id64dLhfH7z0mmcylxuNp starting at 0.0
Now playing: 3dkIE8P7hvl3tHl9KSb6dA starting at 116000.0
Now playing: Joyful activation
Now playing: 3GBApU0NuzH4hKZq4NOSdA starting at 4000.0
Now playing: 6QgjcU0zLnzq5OrUoSZ3OK starting at 120000.0
Now playing: Wonder
Now playing: 5NrV5IwDOFcXHHUFTB8BUy starting at 90000.0
Now playing: 4RW7EZi7SHSLRwvn1Q8Mqj starting at 51000.0
Now playing: Transcendence
Now playing: 4WmB04GBqS4xPMYN9dHgBw starting at 48000.0
Now playing: 3weNRklVDqb4Rr5MhKBR3D starting at 31000.0
Done!
